# Merging

# Índice 
1. [Introduction](#introduction)
2. [Import Materials](#importmaterials)    
    2.1. [Import Libraries](#importlibraries)     
    2.2. [Import the dataset](#importdataset)
3. [Clusterings](#clustering)    


<hr>
<a class="anchor" id="introduction">
    
# 1. Introduction
    
</a>


This notebook analyzes customer data from ABCDEats Inc., a fictional food delivery service, over three months. The goal is to create clusters of customers and make different marketing startegies for each of them....    
**Project by** Dinis Pinto (20240612), Joana Rodrigues (20240603), João Marques (20240656), and Mara Simões (20240326) - **Group 27**.

<hr>
<a class="anchor" id="importmaterials">
    
# 2. Import Materials
    
</a>

<hr>
<a class="anchor" id="importlibraries">
    
## 2.1. Import Libraries
    
</a>

For this project, we'll install some helpful libraries to make analysis easier and add features like better plotting, data handling, and modeling tools. For example, `matplotlib` and `seaborn` allow us to create clear, customizable plots, while `pandas` and `numpy` make data processing smoother.

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pickle
from sklearn.metrics import pairwise_distances
from scipy.cluster.hierarchy import dendrogram
from sklearn.manifold import TSNE
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
#pip install umap-learn
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
from minisom import MiniSom
import umap


<hr>
<a class="anchor" id="importdataset">
    
## 2.2. Import Dataset
    
</a>

### Data in dataset
The different columns in the dataset give out the following information:   
- `customer_id` - Unique identifier for each customer.
- `customer_region` - Geographic region where the customer is located.
- `customer_age` - Age of the customer.
- `vendor_count` - Number of unique vendors the customer has ordered from.
- `product_count` - Total number of products the customer has ordered.
- `is_chain` - Indicates whether the customer’s order was from a chain restaurant.
- `first_order` - Number of days from the start of the dataset when the customer first placed an order.
- `last_order` - Number of days from the start of the dataset when the customer most recently placed an order.
- `last_promo` - The category of the promotion or discount most recently used by the customer.
- `payment_method` - Method most recently used by the customer to pay for their orders.
- `CUI_American`, `CUI_Asian`, `CUI_Chinese`, `CUI_Italian`... - The amount in monetary units spent by the customer from the indicated type of cuisine. 
- `DOW_0` to `DOW_6` - Number of orders placed on each day of the week (0 = Sunday, 6 = Saturday).
- `HR_0` to `HR_23` - Number of orders placed during each hour of the day (0 = midnight, 23 = 11 PM).

In [ ]:
df_behaviour = pickle.load(open("df_behaviour.pkl", 'rb'))
# df_time = pickle.load(open("df_time.pkl", 'rb'))
df_preferences = pickle.load(open("df_preferences.pkl", 'rb'))
# df_preferences_grouped = pickle.load(open("df_preferences_grouped.pkl", 'rb'))

df = pickle.load(open("df_final.pkl", 'rb'))


For the analysis we remove:
- all the DOW_ as the information is not varying a lot and is already present by being divided in days of the week and weekend.
- all the HR_ because the information doesn't vary a lot between them and would lead to a lot of information that can also be seen and analysed through the groups made of hours
- customer_age because as a numeric varible all clusters would have its mean (27.5) as its distribution is normal, so we put them into groups so we can better try to understand which cluster contains each age group, so we transformed age into a categorical column

In [ ]:
df = df.loc[:, ~df.columns.str.startswith(('DOW_', 'HR_', 'customer_age', 'asian_cuisines', 'american_cuisines', 'cafe_desserts'))] 
not_used_columns = [col for col in df.columns if col not in df_preferences.columns and col not in df_behaviour.columns]
categorical_columns = ['customer_region', 'last_promo', 'payment_method', 'age_range', 'customer_city']
numeric_columns = [col for col in df if col not in categorical_columns]

## 3. Optimal clusters for each segmentation
From notebook part 2 and 3 we chose the best clustering approach for each of the segmentation after exploring several different ones.
## 3.1. df_behaviour - 2 clusters with K-Means

In [ ]:
kmclust_behaviour = KMeans(n_clusters=2, init='k-means++', n_init=15, random_state=1)
km_labels = kmclust_behaviour.fit_predict(df_behaviour)

df_concat = pd.concat((df_behaviour, pd.Series(km_labels, name='labels', index=df_behaviour.index)), axis=1)
df_concat.groupby('labels').mean()

## 3.2. df_preferences - ???
To avoid having to repeat code already done we imported the pickle with the clustering already done as SOM is more computationally expensive than K-Means.

In [ ]:
kmclust_pref = KMeans(n_clusters=6, init='k-means++', n_init=15, random_state=1)
km_labels = kmclust_behaviour.fit_predict(df_preferences)

df_concat = pd.concat((df_behaviour, pd.Series(km_labels, name='labels', index=df_preferences.index)), axis=1)
df_concat.groupby('labels').mean()

## 3.3. Merge
In this section we will merge the clusters of the two segmentations.   
The following table shows the 4*3 clusters formed from this.

In [ ]:
# Applying the right clustering (algorithm and number of clusters) for each perspective

behavior_labels = kmclust_behaviour.fit_predict(df_behaviour)
pref_labels = kmclust_pref.fit_predict(df_preferences)

df['behavior_labels'] = behavior_labels
df['preference_labels'] = pref_labels

crosstab_df = pd.crosstab(df['behavior_labels'],
            df['preference_labels'])
crosstab_df

### 3.3.1. Reformulation of clusters
From the previous code we can see that some of the clusters formed have some clusters with few points, this way our goal is to join this points as it doesnt make sense to be doing specific marketing strategies for a very neesh group of people.

In [ ]:
#1. Determining the threshold from which the clusters are joined
threshold = len(df_behaviour) * 0.045  

# Identify clusters smaller than the threshold
to_merge = []
for behavior_label, row in crosstab_df.iterrows():
    for preference_label, count in row.items():
        if count is not None and count < threshold:  # Check for None values
            to_merge.append((behavior_label, preference_label))

# Output results
print("Threshold:", threshold)
print("Clusters to merge:", to_merge)


In [ ]:
df_centroids = df.groupby(['behavior_labels', 'preference_labels'])\
    [numeric_columns].mean()
df_centroids

In [ ]:
# 2. Computing the euclidean distance matrix between the centroids to join the closests
centroid_dists = euclidean = pairwise_distances(df_centroids)

df_dists = pd.DataFrame(
    centroid_dists, 
    columns=df_centroids.index, 
    index=df_centroids.index
)

df_dists

In [ ]:
# 3. Merging each low frequency clustering (source) to the closest cluster (target)
source_target = {}

for clus in to_merge:
    # If the current cluster has not already been used as a target
    if clus not in source_target.values():
        # Find the closest cluster that is not in `to_merge`
        possible_targets = df_dists.loc[clus].sort_values()
        for potential_target in possible_targets.index[1:]:  # Skip self (index[0])
            if potential_target not in to_merge:
                source_target[clus] = potential_target
                break

print(source_target)


In [ ]:
df_ = df.copy()

# Changing the behavior_labels and product_labels based on source_target
for source, target in source_target.items():
    mask = (df_['behavior_labels']==source[0]) & (df_['preference_labels']==source[1])
    df_.loc[mask, 'behavior_labels'] = target[0]
    df_.loc[mask, 'preference_labels'] = target[1]

In [ ]:
pd.crosstab(df_['behavior_labels'],
            df_['preference_labels'])

In [ ]:
# Step 1: Combine behavior_labels and preference_labels into a tuple
df_['behavior_preference_pair'] = list(zip(df_['behavior_labels'], df_['preference_labels']))

# Step 2: Get the unique combinations of (behavior_labels, preference_labels)
unique_combinations = df_['behavior_preference_pair'].unique()

# Step 3: Manually assign a unique value to each unique combination (initial mapping)
initial_mapping = {comb: idx for idx, comb in enumerate(unique_combinations)}


# Step 5: Update the initial mapping based on the merging
# Create a new mapping where the merged clusters will take the same label as their target
merged_mapping = initial_mapping.copy()

# Helper function to find the final target label (propagating merging)
def find_target_label(cluster, source_target, merged_mapping):
    # Traverse the chain until the final target cluster is found
    while cluster in source_target:
        cluster = source_target[cluster]
    # Return the final label of the target
    return merged_mapping.get(cluster, cluster)

# Loop over each source-target pair in the source_target dictionary
for source, target in source_target.items():
    # Find the final target label and propagate it for the source cluster
    merged_mapping[source] = find_target_label(target, source_target, merged_mapping)

# Step 6: Print the full merged mapping
print( merged_mapping)


In [ ]:

# Map the unique combination to its corresponding value
df_['merged_labels'] = df_.apply(
    lambda row: merged_mapping.get(
        (row['behavior_labels'], row['preference_labels']), -1
    ), axis=1
)


In [ ]:
df_.groupby('merged_labels').mean(numeric_only=True)[numeric_columns]

In [ ]:
# Merge cluster contigency table
# Getting size of each final cluster
df_counts = df_.groupby('merged_labels')\
    .size()\
    .to_frame()

# df_counts

In [ ]:
# Getting the product and behavior labels
df_counts = df_counts\
    .rename({v:k for k, v in merged_mapping.items()})\
    .reset_index()

df_counts


In [ ]:
df_counts['behavior_labels'] = df_counts['merged_labels'].apply(lambda x: x[0])
df_counts['preference_labels'] = df_counts['merged_labels'].apply(lambda x: x[1])

df_counts
df_counts.pivot(values=0, index='behavior_labels', columns='preference_labels')

In [ ]:
df_counts.pivot(values=0, index='behavior_labels', columns='preference_labels')

# 4. Cluster analysis and customer profiling
## 4.1.Numerical features
This section will be useful to describe the clusters in the report

In [ ]:
def cluster_profiles(df, label_columns, figsize, cmap="tab10", compare_titles=None):
    """
    Pass df with labels columns of one or multiple clustering labels. 
    Then specify these label columns to perform the cluster profile according to them.
    """
    if compare_titles is None:
        compare_titles = [""] * len(label_columns)
        
    fig, axes = plt.subplots(nrows=len(label_columns), 
                             ncols=2, 
                             figsize=figsize, 
                             constrained_layout=True,
                             squeeze=False)
    
    for ax, label, titl in zip(axes, label_columns, compare_titles):
        # Filtering df to exclude label columns
        drop_cols = [i for i in label_columns if i != label]
        dfax = df.drop(drop_cols, axis=1)
        
        # Get only numeric columns for clustering
        numeric_dfax = dfax.select_dtypes(include=['number'])

        # Getting the cluster centroids and counts
        centroids = numeric_dfax.groupby(by=label, as_index=False).mean()  # Compute mean for numeric columns only
        counts = dfax.groupby(by=label).size().reset_index(name="counts")
        
        # Plotting
        pd.plotting.parallel_coordinates(centroids, 
                                            label, 
                                            color=sns.color_palette(cmap),
                                            ax=ax[0])

        sns.barplot(x=label, 
                    hue=label,
                    y="counts", 
                    data=counts, 
                    ax=ax[1], 
                    palette=sns.color_palette(cmap),
                    legend=False)

        # Setting Layout
        handles, _ = ax[0].get_legend_handles_labels()
        cluster_labels = ["Cluster {}".format(i) for i in range(len(handles))]
        ax[0].annotate(text=titl, xy=(0.95, 1.1), xycoords='axes fraction', fontsize=13, fontweight='heavy') 
        ax[0].axhline(color="black", linestyle="--")
        ax[0].set_title(f"Cluster Means - {len(handles)} Clusters", fontsize=13)
        ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=40, ha='right')
        
        ax[0].legend(handles, cluster_labels,
                     loc='center left', bbox_to_anchor=(1, 0.5), title=label)

        ax[1].set_xticks([i for i in range(len(handles))])
        ax[1].set_xticklabels(cluster_labels)
        ax[1].set_xlabel("")
        ax[1].set_ylabel("Absolute Frequency")
        ax[1].set_title(f"Cluster Sizes - {len(handles)} Clusters", fontsize=13)
        
    plt.suptitle("Cluster Simple Profiling", fontsize=23)
    plt.show()


In [ ]:
df = df_.copy()

# Apply the cluster profiling function
cluster_profiles(
    df=df[numeric_columns + [ 'behavior_labels','preference_labels', 'merged_labels']], 
    label_columns=['behavior_labels','preference_labels', 'merged_labels'], 
    figsize=(28, 13), 
    compare_titles=[ "Behavior clustering", "Preference clustering","Merged clusters"]
)

## 4.2. Categorical columns

In [ ]:
not_used_columns

In [ ]:
# Grouping by the cluster labels and summing for the categorical features
df_cat = df[categorical_columns + ['merged_labels']].groupby(['merged_labels']).apply(lambda x: x.apply(lambda col: col.value_counts().to_dict(), axis=0))

# For visualizing, let's assume you want to create count plots for each categorical feature by cluster
fig, axes = plt.subplots(len(categorical_columns), df['merged_labels'].nunique() + 1, figsize=(16, 4 * len(categorical_columns)), tight_layout=True)

for i, feature in enumerate(categorical_columns):
    for j in range(len(axes[i])):
        ax = axes[i][j]
        if j == 0:
            sns.countplot(data=df, x=feature, order=df[feature].value_counts().index, ax=ax)
            ax.set_title(f"All Data - {feature}")
        else:
            sns.countplot(data=df[df['merged_labels'] == j - 1], x=feature, order=df[feature].value_counts().index, ax=ax)
            ax.set_title(f"Cluster {j - 1}")
        
        ax.tick_params(axis="x", labelrotation=90)
        ax.set_xlabel("")
        ax.set_ylabel("")

plt.suptitle("Categorical Feature Counts by Clusters")
plt.show()


In [ ]:
# Loop through each categorical feature to create both count and percentage stacked bar charts
for feature in not_used_columns:
    # Grouping by 'merged_labels' and the feature, then counting occurrences
    df_cl = df.groupby(['merged_labels', feature])[feature].size().unstack(fill_value=0)
    
    # Create the counts plot
    fig, axes = plt.subplots(1, 2, figsize=(15, 6), tight_layout=True)
    
    # Count plot on the left
    df_cl.plot.bar(stacked=True, ax=axes[0], title=f'{feature} Count Distribution by Clusters')
    axes[0].set_ylabel('Count')
    axes[0].set_xlabel('Cluster Labels')
    axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)  # Optional for better x-axis label visibility
    
    # Calculate the percentage distribution
    df_cl_pct = df_cl.copy()
    
    # Handle missing values by making sure all feature categories are considered
    # Note: We need to make sure each category in 'feature' has the right percentage calculation.
    total_counts = df['merged_labels'].value_counts().sort_index()
    
    for category in df[feature].unique():
        if category in df_cl_pct.columns:
            df_cl_pct[category] = 100 * df_cl_pct[category] / total_counts.values
    
    # Percentage plot on the right
    df_cl_pct.plot.bar(stacked=True, ax=axes[1], title=f'{feature} Percentage Distribution by Clusters')
    axes[1].set_ylabel('Percentage')
    axes[1].set_xlabel('Cluster Labels')
    axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)  # Optional for better x-axis label visibility

    plt.show()


# 5.Assessment of clutering made
## 5.1. T-sne

In [ ]:
two_dim = TSNE(random_state=42).fit_transform(df[numeric_columns])

In [ ]:
pd.DataFrame(two_dim).plot.scatter(x=0, y=1, c=df['merged_labels'], colormap='tab10', figsize=(15,10))
plt.show()

In [ ]:

# Apply UMAP
umap_model = umap.UMAP(random_state=42)
two_dim_umap = umap_model.fit_transform(df[numeric_columns])

# Plot the UMAP results
pd.DataFrame(two_dim_umap).plot.scatter(x=0, y=1, c=df['merged_labels'], colormap='tab10', figsize=(15, 10))
plt.show()


In [ ]:
# Apply PCA
pca_model = PCA(n_components=2, random_state=42)
two_dim_pca = pca_model.fit_transform(df[numeric_columns])

# Plot the PCA results
pd.DataFrame(two_dim_pca).plot.scatter(x=0, y=1, c=df['merged_labels'], colormap='tab10', figsize=(15, 10))
plt.show()



In [ ]:
def get_ss_variables(df):
    """Get the SS for each variable
    """
    ss_vars = df.var() * (df.count() - 1)
    return ss_vars
    

def r2_variables(df, labels):
    """Get the R² for each variable
    """
    sst_vars = get_ss_variables(df)
    ssw_vars = np.sum(df.groupby(labels).apply(get_ss_variables))
    return 1 - ssw_vars/sst_vars

In [ ]:
# We are essentially decomposing the R² into the R² for each variable
r2_variables(df[numeric_columns + ['merged_labels']], 'merged_labels').drop('merged_labels')

In [ ]:
# Preparing the data
X = df[numeric_columns]
y = df.merged_labels

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Fitting the decision tree
dt = DecisionTreeClassifier(random_state=42, max_depth=3)
dt.fit(X_train, y_train)
print("It is estimated that in average, we are able to predict {0:.2f}% of the customers correctly".format(dt.score(X_test, y_test)*100))

In [ ]:
# Assessing feature importance
pd.Series(dt.feature_importances_, index=X_train.columns).sort_values(ascending=False)